In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\VS code programs\\HealthCare\\AI_Medical_Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\VS code programs\\HealthCare\\AI_Medical_Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader (data,
                         glob="*.pdf",
                         loader_cls=PyPDFLoader)

    documents= loader.load()

    return documents

In [7]:
extracted_data= load_pdf_file(data='Data/')

In [8]:
# extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents (extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [11]:
# text_chunks

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings= download_hugging_face_embeddings()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17892\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\LENOVO\Desktop\VS code programs\HealthCare\healthcare_diet_app\backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\Desktop\VS code programs\HealthCare\healthcare_diet_app\backend\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning:

In [18]:
query_result =  embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [20]:
# query_result

In [33]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [35]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name= "medicalbot"

pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'b1e5a661cdc1c7c3f81ff5494b2fa1a7', 'date': 'Mon, 07 Jul 2025 16:59:08 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [26]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [37]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [40]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [44]:
retrieved_docs= retriever.invoke("What is Psoriasis?")

In [45]:
retrieved_docs

[Document(id='3570221f-8380-40f5-9973-982579a83411', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 301.0, 'page_label': '302', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='to benefit the host.\nProtozoa —Animal-like, one-celled organisms,\nsome of which cause diseases in people.\nPsoriasis —A skin disease in which people have\nitchy, scaly, red patches on the skin.\nPurpura—A spotty or patchy purplish rash caused\nby bleeding under the surface of the skin.\n• anticancer drugs\n• medicine for overactive thyroid\n• antiviral drugs such as zidovudine (Retrovir)\nThe list above does not include every medicine that\nmay interact with every antimalarial drug. Be sure to'),
 Document(id='fdf4f212-5019-46da-8efb-f9631e219005', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 248

In [38]:
from langchain_openai import OpenAI
llm= OpenAI(temperature=0.4, max_tokens=500)

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [50]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
response= rag_chain.invoke({"input":"What is Psoriasis?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}